# Задание 1

### Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока). Данные лежат в папке shared. Функция должна быть написана на python. В ходе решения можно тестировать работу функции как на полном датасете, так и на части (сэмпле) данных.

### Импорт данных

In [3]:
import pandas as pd
import time
import numpy as np
import datetime

In [4]:
path_to_file_reg = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-mihajlova/shared/problem1-reg_data.csv'
path_to_file_auth = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-mihajlova/shared/problem1-auth_data.csv'

In [5]:
reg_data = pd.read_csv(path_to_file_reg, sep=';')
auth_data = pd.read_csv(path_to_file_auth, sep=';')

In [6]:
reg_data.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


In [7]:
auth_data.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [8]:
# Проверяем, не является ли столбец reg_ts Unix-временем 
time.gmtime(reg_data.reg_ts[0])

time.struct_time(tm_year=1998, tm_mon=11, tm_mday=18, tm_hour=9, tm_min=43, tm_sec=43, tm_wday=2, tm_yday=322, tm_isdst=0)

###### Исходя из названий столбцов делаем вывод, что 
uid - id пользователя;

reg_ts - время регистрации (в формате Unix)

auth_ts - время использования (в формате Unix)

### Предварительный анализ

In [9]:
reg_data.shape

(1000000, 2)

In [10]:
auth_data.shape

(9601013, 2)

In [11]:
reg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   reg_ts  1000000 non-null  int64
 1   uid     1000000 non-null  int64
dtypes: int64(2)
memory usage: 15.3 MB


In [12]:
auth_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   auth_ts  int64
 1   uid      int64
dtypes: int64(2)
memory usage: 146.5 MB


In [13]:
# Проверка дубликатов
reg_data.duplicated().sum()

0

In [14]:
auth_data.duplicated().sum()

0

In [15]:
# Проверка наличия пустых значений
reg_data.isna().sum()

reg_ts    0
uid       0
dtype: int64

In [16]:
auth_data.isna().sum()

auth_ts    0
uid        0
dtype: int64

Видим, что в данных нет отсутствующих значений и дубликатов. Тип колонок соответствует содержимому.

### Решение

Retention n-ого дня = количество пользоватей в n-ый день / количество зарегистрированных пользователей

Для расчета Retention игроков по дням от даты регистрации игрока воспользуемся методом когортного анализа.
1. Переведем время в дата-фреймах из Unix-формата в формат даты.
2. Разделим пользователей на когорты по дню регистрации и определим количество пользователей (регистраций) в них. Для каждой авторизации пользователя определим, какой это по счету день использования приложения.
3. Для каждой когорты и дня использования рассчитаем количество пользователей, объединим с данными о размере когорты.
4. Рассчитаем retention. Отфильтруем данные по дате и количеству дней. 
5. Сделаем когортную таблицу с данными retention по дате и дням. 

Затем объеденим все шаги внутри функции.

##### 1. Перевод времени в дата-фреймах в дни

In [17]:
reg_data['reg_ts'] = pd.to_datetime(reg_data.reg_ts, unit='s').dt.date
auth_data['auth_ts'] = pd.to_datetime(auth_data.auth_ts, unit='s').dt.date

In [18]:
reg_data.head()

,reg_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,2000-01-13,3
3,2000-05-28,4
4,2000-09-16,5


In [19]:
auth_data.head()

,auth_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,1999-07-25,2
3,1999-07-31,2
4,1999-08-05,2


###### 2. Разделение на когорты

In [20]:
# Определим количество регистраций в каждый из дней
reg_by_days = reg_data.groupby('reg_ts', as_index=False).uid.nunique() \
    .rename(columns={'uid': 'reg_count'})

In [21]:
# Объединим дата-фреймы по общему столбцу 'uid'
data = reg_data.merge(auth_data, on='uid', how='outer')

In [22]:
# Для каждой авторизации определим, какой это день использования приложения (день после регистрации)
data['use_day'] = (data.auth_ts - data.reg_ts).dt.days

In [23]:
data.head()

,reg_ts,uid,auth_ts,use_day
0,1998-11-18,1,1998-11-18,0
1,1999-07-22,2,1999-07-22,0
2,1999-07-22,2,1999-07-25,3
3,1999-07-22,2,1999-07-31,9
4,1999-07-22,2,1999-08-05,14


##### 3. Ипользование приложения по дням и когортам

In [24]:
# Количество пользователей по дням использования и когортам
use_data = data.groupby(['reg_ts', 'use_day'], as_index=False).agg({'uid': 'nunique'})

In [25]:
# Добавим информацию о том, сколько всего регистраций было в когорте
use_data = use_data.merge(reg_by_days, on='reg_ts')

In [26]:
# Переименуем колонки
use_data = use_data.rename(columns={'reg_ts': 'cohort', 'uid': 'user_count'})

In [27]:
use_data.head()

,cohort,use_day,user_count,reg_count
0,1998-11-18,0,1,1
1,1999-07-22,0,1,1
2,1999-07-22,3,1,1
3,1999-07-22,9,1,1
4,1999-07-22,14,1,1


##### 4. Расчет retention 

In [28]:
use_data['retention'] = round((use_data.user_count / use_data.reg_count).mul(100), 2)

In [31]:
# Временной интервал
start_date = '2020-09-10'
end_date = '2020-09-23'

# Перевод строк в объект datetime
start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()


# Количество дней для расчета retention
n = 12

In [32]:
# Фильтрация данных по временному интервалу [start_date, end_date] и количеству дней
filtered_data = use_data.query('@start_date <= cohort <= @end_date and use_day <= @n')

In [33]:
filtered_data.head()

,cohort,use_day,user_count,reg_count,retention
2716123,2020-09-10,0,1609,1609,100.00
2716124,2020-09-10,1,33,1609,2.05
2716125,2020-09-10,2,69,1609,4.29
2716126,2020-09-10,3,77,1609,4.79
2716127,2020-09-10,4,68,1609,4.23


##### 5. Когортная таблица

In [35]:
cohort_table = filtered_data.pivot(index='cohort', columns='use_day', values='retention')

In [36]:
ur_style = (cohort_table
            .style
            .set_caption('User retention by registration days')  # добавляем подпись
            .background_gradient(cmap='cividis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{}%", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту
ur_style

use_day,0,1,2,3,4,5,6,7,8,9,10,11,12
cohort,,,,,,,,,,,,,
2020-09-10,100.0%,2.05%,4.29%,4.79%,4.23%,6.77%,6.28%,7.09%,4.85%,4.54%,5.66%,4.72%,5.22%
2020-09-11,100.0%,1.3%,4.03%,4.53%,5.65%,5.71%,6.7%,6.33%,4.4%,4.65%,5.4%,5.33%,3.29%
2020-09-12,100.0%,1.92%,3.84%,4.71%,4.65%,6.07%,6.57%,5.76%,5.27%,5.14%,5.7%,2.54%,
2020-09-13,100.0%,1.98%,4.08%,4.26%,5.13%,6.55%,6.49%,4.94%,4.7%,4.94%,3.58%,,
2020-09-14,100.0%,2.22%,3.83%,4.51%,5.31%,6.49%,6.98%,6.3%,4.45%,2.9%,,,
2020-09-15,100.0%,2.22%,4.07%,4.56%,6.04%,5.61%,6.9%,5.67%,2.46%,,,,
2020-09-16,100.0%,1.91%,4.25%,4.62%,4.68%,6.77%,6.95%,3.38%,,,,,
2020-09-17,100.0%,1.66%,4.3%,4.55%,5.47%,6.27%,4.24%,,,,,,
2020-09-18,100.0%,1.9%,4.54%,4.23%,5.21%,3.74%,,,,,,,


### Написание функции

In [37]:
def retention(path_to_file_reg, path_to_file_auth, start_date, end_date, n):
    
    '''
    Функция retention рассчитывает retention игроков (по дням от даты регистрации игрока) 
    для заданного отчетного периода [start_date, end_date]. 
    
    Параметры:
    - path_to_file_reg (str) - путь к файлу с данными о времени регистрации;
    - path_to_file_auth (str) - путь к файлу данными о времени захода пользователей в игру;
    - start_date (str) - дата начала отчетного периода;
    - end_date (str) - дата окончания отчетного периода;
    - n (int) - количество дней для расчета retention.
    
    Возвращаемое значение:
    - Таблица со значениям retention по дням в процентах.
    '''
    
    # считывание данных
    reg_data = pd.read_csv(path_to_file_reg, sep=';')
    auth_data = pd.read_csv(path_to_file_auth, sep=';')
    
    # обработка входных данных
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()
    
    # перевод времени из Unix-формата в дни
    reg_data['reg_ts'] = pd.to_datetime(reg_data.reg_ts, unit='s').dt.date
    auth_data['auth_ts'] = pd.to_datetime(auth_data.auth_ts, unit='s').dt.date
    
    # количество регистраций в каждый из дней
    reg_by_days = reg_data.groupby('reg_ts', as_index=False).uid.nunique() \
    .rename(columns={'uid': 'reg_count'})
    
    # разделение на когорты
    data = reg_data.merge(auth_data, on='uid', how='outer')
    data['use_day'] = (data.auth_ts - data.reg_ts).dt.days
    
    # использование приложения по дням и когортам
    use_data = data.groupby(['reg_ts', 'use_day'], as_index=False).agg({'uid': 'nunique'})
    use_data = use_data.merge(reg_by_days, on='reg_ts')
    use_data = use_data.rename(columns={'reg_ts': 'cohort', 'uid': 'user_count'})
        
    # расчет retention
    use_data['retention'] = round((use_data.user_count / use_data.reg_count).mul(100), 2)
    
    # фильтрация данных
    filtered_data = use_data.query('@start_date <= cohort <= @end_date and use_day <= @n')
    
    # когортная таблица
    cohort_table = filtered_data.pivot(index='cohort', columns='use_day', values='retention')
    
    ur_style = (cohort_table
            .style
            .set_caption('User retention by registration days')  # добавляем подпись
            .background_gradient(cmap='cividis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{}%", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту

    return ur_style

In [38]:
print(retention.__doc__)


    Функция retention рассчитывает retention игроков (по дням от даты регистрации игрока) 
    для заданного отчетного периода [start_date, end_date]. 
    
    Параметры:
    - path_to_file_reg (str) - путь к файлу с данными о времени регистрации;
    - path_to_file_auth (str) - путь к файлу данными о времени захода пользователей в игру;
    - start_date (str) - дата начала отчетного периода;
    - end_date (str) - дата окончания отчетного периода;
    - n (int) - количество дней для расчета retention.
    
    Возвращаемое значение:
    - Таблица со значениям retention по дням в процентах.
    


In [39]:
retention(path_to_file_reg, path_to_file_auth, '2020-09-10', '2020-09-23', 12)

use_day,0,1,2,3,4,5,6,7,8,9,10,11,12
cohort,,,,,,,,,,,,,
2020-09-10,100.0%,2.05%,4.29%,4.79%,4.23%,6.77%,6.28%,7.09%,4.85%,4.54%,5.66%,4.72%,5.22%
2020-09-11,100.0%,1.3%,4.03%,4.53%,5.65%,5.71%,6.7%,6.33%,4.4%,4.65%,5.4%,5.33%,3.29%
2020-09-12,100.0%,1.92%,3.84%,4.71%,4.65%,6.07%,6.57%,5.76%,5.27%,5.14%,5.7%,2.54%,
2020-09-13,100.0%,1.98%,4.08%,4.26%,5.13%,6.55%,6.49%,4.94%,4.7%,4.94%,3.58%,,
2020-09-14,100.0%,2.22%,3.83%,4.51%,5.31%,6.49%,6.98%,6.3%,4.45%,2.9%,,,
2020-09-15,100.0%,2.22%,4.07%,4.56%,6.04%,5.61%,6.9%,5.67%,2.46%,,,,
2020-09-16,100.0%,1.91%,4.25%,4.62%,4.68%,6.77%,6.95%,3.38%,,,,,
2020-09-17,100.0%,1.66%,4.3%,4.55%,5.47%,6.27%,4.24%,,,,,,
2020-09-18,100.0%,1.9%,4.54%,4.23%,5.21%,3.74%,,,,,,,
